In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install --quiet scvi-colab
from scvi_colab import install
install()

/home/mo/miniconda3/envs/ivelo/lib/python3.8/site-packages/scvi_colab/_core.py:41: UserWarning: 
                Not currently in Google Colab environment.

                Please run with `run_outside_colab=True` to override.

                Returning with no further action.
                
  warn(


In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import torch
import os
import pickle
import mplscience
import scarches as sca
import matplotlib.pyplot as plt
import seaborn as sns
from velovi import VELOVI
# import ontology modules
from velovi.ontobj import *
from velovi._model import *


 captum (see https://github.com/pytorch/captum).
INFO:lightning_lite.utilities.seed:Global seed set to 0


In [4]:
#Choose dataset
dataset = "pancreas"

if dataset == "pbmc":
    cell_column = 'celltype'
elif dataset == "pancreas":
    cell_column = 'clusters'
elif dataset == "dentategyrus":
    cell_column = 'clusters'
elif dataset == "hindbrain":
    cell_column = 'Celltype'

#Choose model to train
model = "hivelo"

if model =="hivelo":
    use_ontology=True

In [5]:
if model =="hivelo":
    adata_file = "data/" + dataset + "/" + "processed_ontology.h5ad"
else:
    adata_file = "data/" + dataset + "/" + dataset + "_data_annotations.h5ad" #preprocessed data
adata_output = "data/" + dataset + "/" + model + "/" + "data_output.h5ad" #path so save output adata after training model
metrics_dict = "output/" + dataset + "/" + model + "/latent_analysis.pkl"

In [6]:
adata=sc.read(adata_file)

### Create ontology object

In [7]:
# initialize the Ontobj
# the description should be an identifier, e.g. the ontology used, here: PWO (Pathway Ontology)
pwo = Ontobj()

In [8]:
# initialize our ontology
# obo: path to an obo file
# gene_annot: path to a tab separated file with two columns: Genes and Ontology IDs
top_thresh=1000
bottom_thresh=30
path_ontology = "data/ontology/"
pwo.initialize_dag(obo=path_ontology + 'pw.obo',
                   gene_annot=path_ontology + 'gene_term_mapping.txt')

# trim the ontology
pwo.trim_dag(top_thresh=top_thresh, 
             bottom_thresh=bottom_thresh)

# create masks for decoder initialization
pwo.create_masks(top_thresh=top_thresh,
                 bottom_thresh=bottom_thresh)

# match a dataset to the ontology
# expr_path: path to the dataset (either h5ad)
pwo.match_dataset(expr_data = adata_file,
                  name=dataset)                                    

In [9]:
adata=pwo.data[str(top_thresh) + '_' + str(bottom_thresh)][dataset].copy()
vae=VELOVI.load("trained_models/" + dataset + "/" + model, adata)

INFO     File trained_models/pancreas/hivelo/model.pt already downloaded                                           
Last Decoder layer: identity
